In [78]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import os, sys, fnmatch
import cv2, time
import numpy as np
import argparse, pprint


from matplotlib import pyplot as plt
import matplotlib
from matplotlib.ticker import NullFormatter
from matplotlib.transforms import Bbox
import matplotlib.gridspec as gridspec

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

from src.UVMappingPipeline import UVMappingPipeline
from src.VBOATS import VBOATS
from utils.sys_utils import *
from utils.seg_utils import *
from utils.uv_mapping_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
pipe = UVMappingPipeline()
vboat = VBOATS()

In [18]:
def pplot(img,num = 0):
    plt.figure(num)
    plt.imshow(img)
    plt.subplots_adjust(wspace=0.0,hspace=0.0,left=0.0,right=1.0,top=1.0, bottom=0.0)
    plt.show()

In [ ]:
# tdir = "/home/hunter/data/images/obstacles"
tdir = "/home/hunter/data/images/test_cases"
# tdir = "/home/hyoung/data/images/obstacles"
# tdir = homeDir + "/test/iai/obstacles"

found_imgs, _ = grab_dir_images(tdir)

In [ ]:
%reload_ext autoreload
plt.close('all')

p = interactive(pipe.pipeline,
    _img = widgets.Select(
                        options=sorted(found_imgs),
                        value=str(sorted(found_imgs)[0]),
                        description='File:',
                        layout=Layout(width='100%')
                    ),
    line_method = {'Standard Hough Transform':0,'Probabilistic Hough Transform':1},
    input_method = {'Classic Thresholded':0,'Morphed Thresholded':1,'Canny Edges':2},
    e1 = (1,255,1), e2 = (1,255,1),
    ang = (0,360,1), rho = (1,1000,1),
    minLineLength = (0,255,1), maxLineGap = (0,255,1),
    houghThresh = (0,255,1), greyThreshU = (0,255,1), greyThreshV = (0,255,1), cnt_thresh = (0,5000,100),
    show_helpers = True, use_umap = True, flip_thresh_bin = False, norm_grey_thresh = (0.0,1.0,0.01)
)


row1 = HBox([p.children[0]])
row2 = HBox([p.children[1], p.children[2]], layout = Layout(flex_flow='row wrap'))
row3 = HBox([p.children[3], p.children[4], p.children[9]], layout = Layout(flex_flow='row wrap'))
row4 = HBox([p.children[5], p.children[6], p.children[7], p.children[8]], layout = Layout(display='flex',flex_flow='row'))

row5 = HBox([p.children[10], p.children[11], p.children[12], ], layout = Layout(flex_flow='row wrap'))
row6 = HBox([p.children[16]], layout = Layout(flex_flow='row wrap'))
row7 = HBox([p.children[13], p.children[14], p.children[15]], layout = Layout(flex_flow='row wrap'))
controls = VBox([row1,row2,row3,row4,row5,row6,row7], layout = Layout(display='flex'))
output = p.children[-1]
display(VBox([controls, output]))

In [ ]:
tmp1 = np.copy(pipe.vmap); tmp0 = cv2.cvtColor(tmp1,cv2.COLOR_GRAY2BGR)

normImg = (tmp1 - np.mean(tmp1))/np.std(tmp1)

dmax_norm = np.max(normImg)
dmin_norm = np.min(normImg)
test = cv2.normalize(normImg,None,0,1,cv2.NORM_MINMAX)
m = 255.0/(dmax_norm-dmin_norm)
print(dmin_norm,dmax_norm,m)
tmp = cv2.convertScaleAbs(test, test)#, m)#,np.mean(tmp))
print(np.min(tmp),np.max(tmp),np.mean(tmp))

fig1 = plt.figure(6)
fig1.canvas.set_window_title('Custom Filter Input')
plt.imshow(pipe.vmap,cmap='gray')
plt.show()

iters = 3; kSz = (1,6)
thresh = [5, 10, 20, 30]
filters = [cv2.THRESH_BINARY,cv2.THRESH_BINARY_INV,cv2.THRESH_TRUNC+cv2.THRESH_BINARY_INV,cv2.THRESH_TOZERO]
kernelC = cv2.getStructuringElement(cv2.MORPH_CROSS,kSz)
kernelE = cv2.getStructuringElement(cv2.MORPH_RECT,(1,15))

_, grey = cv2.threshold(tmp1,5,255,filters[0])#+cv2.THRESH_TRUNC)

# gradient = cv2.morphologyEx(grey, cv2.MORPH_CLOSE, kSz)
gradient = cv2.morphologyEx(grey, cv2.MORPH_OPEN, kSz)

# tmpKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,kSz)
tmpKernel = cv2.getStructuringElement(cv2.MORPH_RECT,kSz)

gradient2 = cv2.erode(tmp0,kernelE,iterations = 1)
gradient2 = cv2.dilate(gradient2,tmpKernel,iterations = 1)
# preMorph = cv2.erode(grey,kernelE,iterations = 3)
# preMorph = cv2.dilate(grey,tmpKernel,iterations = 3)


# # morph = cv2.morphologyEx(grey,cv2.MORPH_CLOSE,(1,2), iterations = 2)
# morph = cv2.morphologyEx(grey,cv2.MORPH_OPEN,(2,3), iterations = 1)

# _,preMorph = cv2.threshold(preMorph,1,255,cv2.THRESH_BINARY_INV)
# tmpKernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,15))
# preMorph = cv2.dilate(preMorph,tmpKernel,iterations = 3)
# tmpKernel = cv2.getStructuringElement(cv2.MORPH_RECT,(10,3))
# preMorph = cv2.erode(preMorph,kernelE,iterations = 2)
# print(preMorph.shape)

# mask = cv2.cvtColor(preMorph,cv2.COLOR_BGR2GRAY)
# res = cv2.bitwise_and(tmp,tmp, mask=mask)
_, res = cv2.threshold(gradient2,5,255,cv2.THRESH_BINARY)

out = res

fig2 = plt.figure(7)
fig2.canvas.set_window_title('Custom Filter Output')
plt.imshow(out,cmap='gray')
plt.show()

In [ ]:
print("""
POTENTIALLY GOOD VMAP FILTERING
""")

tmp1 = np.copy(pipe.vmap); tmp0 = cv2.cvtColor(tmp1,cv2.COLOR_GRAY2BGR)

kSz = (1,6)
kernelE = cv2.getStructuringElement(cv2.MORPH_RECT,(1,15))
_, grey = cv2.threshold(tmp1,5,255,cv2.THRESH_BINARY)

gradient = cv2.morphologyEx(grey, cv2.MORPH_OPEN, kSz)
tmpKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,kSz)

gradient2 = cv2.erode(tmp0,kernelE,iterations = 1)
gradient2 = cv2.dilate(gradient2,tmpKernel,iterations = 1)
_, res = cv2.threshold(gradient2,5,255,cv2.THRESH_BINARY)

out = res

In [ ]:
# print(pipe.disparityBounds[obsId],pipe.xBounds[obsId])

tmp = np.copy(pipe.greyV)
tmp2 = np.copy(pipe.img)
tmp2 = cv2.cvtColor(tmp2,cv2.COLOR_GRAY2BGR)
nObs = len(pipe.disparityBounds)
print(nObs)
b = 2; d = np.ceil(nObs / float(b))

plt.close(7)
fig, axes = plt.subplots(nrows=b, ncols=int(d), num=7)
obstacleImgs = []
for i in range(nObs):
    tmp2 = np.copy(pipe.img)
    tmp2 = cv2.cvtColor(tmp2,cv2.COLOR_GRAY2BGR)
    try:
        xs = pipe.xBounds[i]
        ys,disp = pipe.find_obstacles(tmp,pipe.disparityBounds[i], _threshold=30, lower_threshold=1, verbose = True)
        cv2.rectangle(tmp2,(xs[0],ys[0]),(xs[1],ys[-1]),(150,0,0),1)
    except:
        pass
    fig.axes[i].imshow(disp)
    obstacleImgs.append(tmp2)
plt.show()

In [ ]:
os.path.join()

In [ ]:
im = np.copy(pipe.img)
norm = (im - np.mean(im))/np.std(im)
h, w = norm.shape[:2]

umapOrig, vmapOrig,_ = pipe.get_uv_map(im,get_overlay=False)

plt.figure(0)
plt.imshow(im, cmap='gray')
plt.show()
plt.figure(1)
plt.imshow(norm, cmap='gray')
plt.show()

In [82]:
# tdir = "/home/hyoung/devel/vision_playground/test/iai/obstacles"
# tdir = "/home/hunter/data/images/test_cases"
tdir = "/home/hyoung/Documents/rosbags/extracted/depth"

In [114]:
found_imgs, _ = grab_dir_images(tdir)
_imgs = widgets.Select(
                        options=sorted(found_imgs),
                        value=str(sorted(found_imgs)[0]),
                        description='File:',
                        layout=Layout(width='100%')
                    )
# display(_imgs)

def tempInteract(rng, lbl):
    print(str(lbl), rng)
    
testSct = widgets.IntRangeSlider(
    value=[5, 7],
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

p = interactive(tempInteract, rng = testSct, lbl = "Hey")

controls = HBox([p.children[0],p.children[1]], layout = Layout(display='flex'))
output = p.children[-1]
tabs = widgets.Tab([controls, output])
# disp = VBox(tabs)

display(tabs)

VGFiKGNoaWxkcmVuPShIQm94KGNoaWxkcmVuPShJbnRSYW5nZVNsaWRlcih2YWx1ZT0oNSwgNyksIGNvbnRpbnVvdXNfdXBkYXRlPUZhbHNlLCBkZXNjcmlwdGlvbj11J1Rlc3Q6JywgbWF4PTHigKY=


In [104]:
def handle_control_change(change):
    output = p.children[-1]
    print(change['new'])
    display(output)
    
controls.observe(handle_control_change, names='value')

In [90]:
caption = widgets.Label(value='The values of range1 and range2 are synchronized')
slider = widgets.IntSlider(min=-5, max=5, value=1, description='Slider')

def handle_slider_change(change):
    caption.value = 'The slider value is ' + (
        'negative' if change.new < 0 else 'nonnegative'
    )

slider.observe(handle_slider_change, names='value')

display(caption, slider)

Label(value=u'The values of range1 and range2 are synchronized')

IntSlider(value=1, description=u'Slider', max=5, min=-5)

In [97]:
caption = widgets.Label(value='Changes in source values are reflected in target1')
source, target1 = widgets.IntSlider(description='Source'),\
                  widgets.IntSlider(description='Target 1')
dl = widgets.dlink((source, 'value'), (target1, 'value'))
display(caption, source, target1)

Label(value=u'Changes in source values are reflected in target1')

IntSlider(value=0, description=u'Source')

IntSlider(value=0, description=u'Target 1')

In [ ]:
from utils.uv_mapping_utils import *
from utils.seg_utils import *

_img = _imgs.value
img = cv2.imread(_img, cv2.IMREAD_GRAYSCALE)

kernelI = cv2.getStructuringElement(cv2.MORPH_RECT,(10,10))
img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernelI)

# umap, vmapMasked, img, obs, nObs, pxls,vmap,mask, maskInv, dt = uv_pipeline(_img, flag_displays=True)

umap1, vmap1,_ = get_uv_map(img)
umap2 = umapping(img);              vmap2 = vmapping(img);               overlay = make_uv_overlay(img,umap2,vmap2)
h,w = umap1.shape[:2]

raw_umap = np.copy(umap1)

# =================================================================================
# =================================================================================
# =================================================================================

dead_y = 10
deadzoneU = raw_umap[1:dead_y+1, :]
blankStrip = np.zeros((h-dead_y,w),dtype=np.uint8)

print(np.max(deadzoneU))
_, deadzoneU = cv2.threshold(deadzoneU, 95, 255,cv2.THRESH_BINARY)
cv2.rectangle(raw_umap,(0,0),(raw_umap.shape[1],dead_y),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_umap,(0,raw_umap.shape[0]-dead_y),(raw_umap.shape[1],raw_umap.shape[0]),(0,0,0), cv2.FILLED)


deadzone_mask = np.concatenate((deadzoneU,blankStrip), axis=0)
try: deadzone_mask = cv2.cvtColor(deadzone_mask, cv2.COLOR_GRAY2BGR)
except: print("[WARNING] ------------  Unnecessary Deadzone Image Color Converting")

kernelD = cv2.getStructuringElement(cv2.MORPH_RECT,(50,5))
deadzone_mask = cv2.morphologyEx(deadzone_mask, cv2.MORPH_CLOSE, kernelD)

try: raw_umap = cv2.cvtColor(raw_umap, cv2.COLOR_GRAY2BGR)
except: print("[WARNING] ------------  Unnecessary raw_umap Image Color Converting")
umap = cv2.addWeighted(raw_umap, 1.0, deadzone_mask, 1.0, 0)

try: umap = cv2.cvtColor(umap, cv2.COLOR_BGR2GRAY)
except: print("[WARNING] ------------  Unnecessary stripU1 Image Color Converting")

stripsU = strip_image(umap, nstrips=4, horizontal_strips=False)

strips = np.copy(stripsU)
print(stripsU.shape, strips.shape)

# _, stripU1 = cv2.threshold(stripsU[0], 7, 255,cv2.THRESH_BINARY)
# _, stripU2 = cv2.threshold(stripsU[1], 25, 255,cv2.THRESH_BINARY)
# _, stripU3 = cv2.threshold(stripsU[2], 30, 255,cv2.THRESH_BINARY)
# _, stripU4 = cv2.threshold(stripsU[3], 40, 255,cv2.THRESH_BINARY)


# fig, axes = plt.subplots(1,4)
# axes[0].imshow(stripsU[0])
# axes[1].imshow(stripsU[1])
# axes[2].imshow(stripsU[2])
# axes[3].imshow(stripsU[3])
# fig.show()

color = (255,0,255)
borderb = np.ones((1,w,3),dtype=np.uint8); borderb[:] = color
borders = np.ones((h,1,3),dtype=np.uint8); borders[:] = color
# newUdisp = np.concatenate((stripU1,borderb,stripU2,borderb,stripU3,borderb,stripU4), axis=0)
# newUdisp = np.concatenate((stripU1,borders,stripU2,borders,stripU3,borders,stripU4), axis=1)


# cv2.rectangle(raw_vmap,(0,0),(dead_x, raw_vmap.shape[0]),(0,0,0), cv2.FILLED)
# cv2.rectangle(raw_vmap,(raw_vmap.shape[1]-dead_x,0),(raw_vmap.shape[1],raw_vmap.shape[0]),(0,0,0), cv2.FILLED)
# newVdisp = np.concatenate((stripV1,borders,stripV2,borders,stripV3,borders,stripV4), axis=1)

_, tmp = cv2.threshold(umap, 10, 255,cv2.THRESH_TOZERO)

plt.figure(8)
plt.imshow(img)
plt.show()

plt.figure(10)
plt.imshow(umap)
plt.show()

plt.figure(11)
plt.imshow(vmap1)
plt.show()


plt.figure(12)
plt.imshow(umap2)
plt.show()

plt.figure(13)
plt.imshow(vmap2)
plt.show()

In [ ]:
norm = (vmap1 - np.mean(vmap1))/ np.std(vmap1)

plt.figure(14)
plt.imshow(norm)
plt.show()

In [ ]:
h,w = vmap.shape[:2]
# black = np.zeros((h,w,3),dtype=np.uint8)
black = np.copy(vmap)
black = cv2.cvtColor(black,cv2.COLOR_GRAY2BGR)

print(pxls[-1,0],pxls)

# ploty = np.linspace(pxls[0,1], pxls[-1,1])
plotx = np.linspace(pxls[0,0], pxls[-1,0])
try: # Try fitting polynomial nonzero data
    fit = np.poly1d(np.polyfit(pxls[:,0],pxls[:,1], 2))
    # Generate x and y values for plotting
#     plotx = fit(ploty)
    ploty = fit(plotx)
except:
    print("ERROR: Function 'polyfit' failed!")
    fit = [0, 0]; plotx = [0, 0]
# xs = np.asarray(ploty,dtype=np.int32)
# ys = np.asarray(plotx,dtype=np.int32)

xs = np.asarray(plotx,dtype=np.int32)
ys = np.asarray(ploty,dtype=np.int32)

pts = np.vstack(([xs],[ys])).transpose()
cv2.polylines(black, [pts], 0, (255,0,255),5)
plt.figure(9)
plt.imshow(black)
# plt.plot(pxls[:,0],pxls[:,1])
# plt.plot(ploty,-plotx)

plt.show()

In [ ]:
def tester2(_img, e1=1, e2=5,tmpThresh=3,greyThreshU=7,greyThreshV=5,cnt_thresh=100.0,
                 show_helpers=True,use_umap=True, flip_thresh_bin=True, norm_grey_thresh=0.5):
    img = cv2.imread(_img, cv2.IMREAD_GRAYSCALE)
    pipe.img = img
    plt.figure(0)
    plt.imshow(img)
    plt.show()

    if(flip_thresh_bin):
        umap, vmap, overlay = pipe.get_uv_map(img, get_overlay=True)
        dead_x = 2; dead_y = 10

        cv2.rectangle(umap,(0,0),(umap.shape[1],dead_y),(0,0,0), cv2.FILLED)
        cv2.rectangle(umap,(0,umap.shape[0]-dead_y),(umap.shape[1],umap.shape[0]),(0,0,0), cv2.FILLED)

        cv2.rectangle(vmap,(0,0),(dead_x, vmap.shape[0]),(0,0,0), cv2.FILLED)
        cv2.rectangle(vmap,(vmap.shape[1]-dead_x,0),(vmap.shape[1],vmap.shape[0]),(0,0,0), cv2.FILLED)
    else:
        umap = umapping(img)
        vmap = vmapping(img)
        overlay = make_uv_overlay(img,umap,vmap)


    h,w = umap.shape[:2]
    clone = np.copy(umap)
#     cv2.rectangle(clone,(0,0),(w,10),(0,0,0), cv2.FILLED)
#     cv2.rectangle(clone,(0,h-10),(w,h),(0,0,0), cv2.FILLED)
    print(np.max(clone), np.min(clone))



    _,tmp = cv2.threshold(clone,greyThreshU,np.max(clone),cv2.THRESH_BINARY)
    opening = tmp
#     _,tmp = cv2.threshold(clone,greyThreshU,np.max(clone),cv2.THRESH_TOZERO)

#     cv2.rectangle(tmp,(0,0),(w,10),(0,0,0), cv2.FILLED)
#     cv2.rectangle(tmp,(0,h-10),(w,h),(0,0,0), cv2.FILLED)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(e1,e2))
#     opening = cv2.morphologyEx(tmp, cv2.MORPH_CLOSE, kernel)
#     _,opening = cv2.threshold(opening,tmpThresh,np.max(clone),cv2.THRESH_BINARY)

    _,greyV = cv2.threshold(vmap,greyThreshV,np.max(vmap),cv2.THRESH_BINARY)

    _, contours, hierarchy = cv2.findContours(opening,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(cnt) for cnt in contours]
    perimeters = [cv2.arcLength(cnt,True) for cnt in contours]

    filtered_cnts = [cnt for cnt in contours if cv2.arcLength(cnt,True) >= cnt_thresh]
    filtered_perimeters = [cv2.arcLength(cnt,True) for cnt in filtered_cnts]
    print("Filtered Contour Perimeters:",filtered_perimeters)

    display = cv2.cvtColor(opening, cv2.COLOR_GRAY2BGR)
    for cnt in filtered_cnts:
        cv2.drawContours(display, [cnt], 0, (255,0,0), 2)

    cnts, limits, ellipses, pts, centers = pipe.extract_contour_x_bounds(filtered_cnts)

    nObs = len(pipe.disparityBounds)
    obsImgs = []
    disp = np.copy(img)
    display_obstacles = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)

    for i in range(nObs):
        tmp = np.copy(img)
        tmp = cv2.cvtColor(tmp,cv2.COLOR_GRAY2BGR)
        try:
            xs = pipe.xBounds[i]
            ys,disp = pipe.find_obstacles(greyV,pipe.disparityBounds[i], _threshold=30, lower_threshold=1, verbose = True, display=greyV)
            cv2.rectangle(tmp,(xs[0],ys[0]),(xs[1],ys[-1]),(150,0,0),1)
            cv2.rectangle(display_obstacles,(xs[0],ys[0]),(xs[1],ys[-1]),(150,0,0),1)
        except: pass
        obsImgs.append(tmp)


    plt.figure(1)
    plt.imshow(opening)
    plt.show()

    plt.figure(2)
    plt.imshow(overlay)
    plt.show()

    plt.figure(3)
    plt.imshow(display)
    plt.show()

    plt.figure(4)
    plt.imshow(display_obstacles)
    plt.show()

    plt.figure(6)
    plt.imshow(greyV)
    plt.show()

In [80]:
vboat.dead_x = 0
vboat.dead_y = 3
vboat.dH = 3
vboat.dW = 0

In [76]:
def thresholding(_img, e1=10, e2=5,tmpThresh=70
                 
                 ,greyThreshU=11,greyThreshV=7,cnt_thresh=100.0,
                 show_helpers=False,use_umap=True, flip_thresh_bin=True, norm_grey_thresh=0.15
):
    if(show_helpers):
        vboat.pipeline(_img, threshU1=greyThreshU,threshU2=greyThreshV, threshV2=tmpThresh)
        display_obstacles = cv2.cvtColor(vboat.img, cv2.COLOR_GRAY2BGR)

        for ob in vboat.obstacles:
            cv2.rectangle(display_obstacles,ob[0],ob[1],(150,0,0),1)
        plt.figure(9)
        plt.imshow(display_obstacles)
        plt.show()
    else:
        vboat.dW = e1
        vboat.dH = e2
        vboat.v0Thresh = norm_grey_thresh
        vboat.pipelineTest(_img, threshU1=greyThreshU,threshU2=greyThreshV, threshV2=tmpThresh)
        display_obstacles = cv2.cvtColor(vboat.img, cv2.COLOR_GRAY2BGR)

        for ob in vboat.obstacles:
            cv2.rectangle(display_obstacles,ob[0],ob[1],(0,0,255),1)
        
        overlay = make_uv_overlay(display_obstacles,vboat.umap_deadzoned,vboat.vmap_raw)
        overlay = cv2.cvtColor(overlay,cv2.COLOR_BGR2RGB)
        pplot(overlay,'Overlay')

        dispU1,dispU2,compU = vboat.umap_displays()
        dispV1,dispV2,compV = vboat.vmap_displays()

        dispU2 = cv2.cvtColor(dispU2,cv2.COLOR_BGR2RGB)
        pplot(dispU2,'Umap Display')
        dispV2 = cv2.cvtColor(dispV2,cv2.COLOR_BGR2RGB)
        pplot(dispV2,'Vmap Display')
        # tester2(_img,e1,e2,tmpThresh,greyThreshU,greyThreshV,cnt_thresh,show_helpers,use_umap,flip_thresh_bin, norm_grey_thresh)
    
        

In [77]:

# found_imgs, _ = grab_dir_images("/home/hunter/data/images/test_cases")
# found_imgs, _ = grab_dir_images("/home/hunter/devel/vision_playground/test/frames")
# found_imgs, _ = grab_dir_images("/home/hunter/data/images/obstacles")
# found_imgs, _ = grab_dir_images("/home/hunter/data/rosbags/extracted/depth")

# found_imgs, _ = grab_dir_images("/home/hyoung/Documents/rosbags/extracted/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/devel/vision_playground/test/iai/obstacles")

found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/tmp/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/distance_test/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/angle_test/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/misc_test/depth")

ttt = interactive(thresholding,_img = widgets.Select(
                        options=sorted(found_imgs),
                        value=str(sorted(found_imgs)[0]),
                        description='File:',
                        layout=Layout(width='100%')
                    ), e1 = (1,255,1), e2 = (1,255,1),
    tmpThresh = (0,255,1), greyThreshU = (0.0,255.0,1), greyThreshV = (0,255,1), cnt_thresh = (0,5000,100),
    show_helpers = False, use_umap = True, flip_thresh_bin = False, norm_grey_thresh = (0.0,1.0,0.01)
)
display(ttt)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KFNlbGVjdChkZXNjcmlwdGlvbj11J0ZpbGU6JywgbGF5b3V0PUxheW91dCh3aWR0aD11JzEwMCUnKSwgb3B0aW9ucz0oJy9ob21lL2h5b3VuZy9kYXRhL2TigKY=


In [44]:
tImg = np.copy(vboat.img)
umap = vboat.umap_raw
vmap = vboat.vmap_raw
deadu = vboat.umap_deadzoned
pv = vboat.vmap_filtered
pu = vboat.umap_processed

# pplot(tImg,'Input')
pplot(umap,'Raw Umap')
# pplot(vmap,'Raw Vmap')
pplot(deadu,'Deadzoned Umap')
pplot(pv,'Processed vmap')



# mn = np.mean(tImg)
# st = np.std(tImg[tImg > 10])
# norm = (tImg - mn)/st

# norm = np.expand_dims(norm,axis=-1)

# norm = cv2.normalize(norm,None,0.0,255.0,cv2.NORM_MINMAX)
# norm = np.asarray(norm, dtype=np.uint8)
# nUmap, nVmap,_ = vboat.get_uv_map(norm)

# sub = nUmap - umap 
# pplot(sub,'Sub')

# copy = vmap.copy()
# # copy = cv2.cvtColor(copy,cv2.COLOR_GRAY2BGR)
# copy = np.float32(copy)
# criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
# K = 150
# ret,label,center=cv2.kmeans(copy,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

# # Now convert back into uint8, and make original image
# center = np.uint8(center)
# res = center[label.flatten()]
# res2 = res.reshape((vmap.shape))
# pplot(res2,'kmeans')

# nUmap = nUmap**2
# nUmap[nUmap < np.max(nUmap*0.3)] = 0

# pplot(nUmap,'Norm Umap')

# pplot(vboat.umap_processed,'Processed Umap')

# _, tmpU = cv2.threshold(vmap, 10, 255,cv2.THRESH_TOZERO)
# tmpU = cv2.equalizeHist(tmpU)

# pplot(tmpU,1)

In [43]:
dispU1,dispU2,compU = vboat.umap_displays()
dispV1,dispV2,compV = vboat.vmap_displays()

# plt.figure(6)
# plt.imshow(vboat.img)
# plt.show()

dispU2 = cv2.cvtColor(dispU2,cv2.COLOR_BGR2RGB)
plt.figure(0)
plt.imshow(dispU2)
plt.show()

# plt.figure(1)
# plt.imshow(dispU2)
# plt.show()

plt.figure(2)
plt.imshow(vboat.vmap_processed**2)
plt.show()

dispV2 = cv2.cvtColor(dispV2,cv2.COLOR_BGR2RGB)
plt.figure(3)
plt.imshow(dispV2)
plt.show()

# plt.figure(4)
# plt.imshow(dispV2)
# plt.show()

# plt.figure(5)
# plt.imshow(compV)
# plt.show()

In [ ]:
tImg = np.copy(vboat.img)

raw_umap, raw_vmap, dt = vboat.get_uv_map(tImg)

# =========================================================================
dead_y = vboat.dead_y; dead_x = vboat.dead_x

deadzoneU = raw_umap[1:dead_y+1, :]
_, deadzoneU = cv2.threshold(deadzoneU, 80, 255,cv2.THRESH_BINARY)

cv2.rectangle(raw_umap,(0,0),(raw_umap.shape[1],dead_y),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_umap,(0,raw_umap.shape[0]-dead_y),(raw_umap.shape[1],raw_umap.shape[0]),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_vmap,(0,0),(dead_x, raw_vmap.shape[0]),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_vmap,(raw_vmap.shape[1]-dead_x,0),(raw_vmap.shape[1],raw_vmap.shape[0]),(0,0,0), cv2.FILLED)
# =========================================================================

threshs = [11, 10, 30, 40,40,40]
stripsPu = []
stripsU = strip_image(raw_umap, nstrips=len(threshs))

strU1 = stripsU[0]
strU2 = stripsU[1]

for i, strip in enumerate(stripsU):
#     _, tmpStrip = cv2.threshold(strip, threshs[i], 255,cv2.THRESH_BINARY)
    _, tmpStrip = cv2.threshold(strip, threshs[i], 255,cv2.THRESH_TOZERO)
    stripsPu.append(tmpStrip)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(15,2))
stripsPu[0] = cv2.morphologyEx(stripsPu[0], cv2.MORPH_CLOSE, kernel)

tK = np.ones((3,2),np.uint8)
copy = stripsPu[1].copy()

stripsPu[1] = cv2.morphologyEx(stripsPu[1], cv2.MORPH_CLOSE, kernel)

# stripsPu[1] = cv2.morphologyEx(stripsPu[1], cv2.MORPH_OPEN, (2,2))
# stripsPu[1] = cv2.blur(stripsPu[1],(2,2))
tmpp = cv2.morphologyEx(stripsPu[1], cv2.MORPH_GRADIENT, kernel)
# stripsPu[1] = cv2.equalizeHist(stripsPu[1])
# stripsPu[2] = cv2.morphologyEx(stripsPu[2], cv2.MORPH_OPEN, kernel)
pplot(tmpp, 'tmpp')
stripsPu[1] = cv2.morphologyEx(stripsPu[1], cv2.MORPH_BLACKHAT, kernel)

# ============================ CRAP ============================
kernel = np.ones((4,4),np.uint8)

_, tmpStrip = cv2.threshold(stripsPu[1], 1, 255,cv2.THRESH_BINARY)
# _, tmpStrip2 = cv2.threshold(stripsPu[1], 1, 255,cv2.THRESH_BINARY_INV)
# dist_transform = cv2.distanceTransform(stripsPu[1],cv2.DIST_L2,5)
dist_transform = cv2.distanceTransform(tmpStrip,cv2.DIST_L2,5)
# dist_transform2 = cv2.distanceTransform(tmpStrip2,cv2.DIST_L2,5)
# pplot(dist_transform, 'dist_transform')
# pplot(dist_transform2, 'dist_transform2')

# erode = cv2.dilate(dist_transform2,kernel,iterations=5)
erode = cv2.erode(dist_transform,kernel,iterations=1)
# stripsPu[1] = np.uint8(erode)

# dist_transform = cv2.distanceTransform(stripsPu[1],cv2.DIST_L2,5)
# erode = cv2.erode(dist_transform,kernel,iterations=2)
# _, sure_fg = cv2.threshold(erode,0.1*erode.max(),255,0)

# sure_bg = cv2.dilate(sure_fg,kernel,iterations=1)
# unknown = cv2.subtract(sure_bg,sure_fg)

# # pplot(dist_transform, 'dist_transform')
# # pplot(sure_fg, 'sure_fg')
# # pplot(erode, 'erode')

# sure_fg = np.uint8(sure_fg)
# _, markers = cv2.connectedComponents(sure_fg)


# markers = markers+1
# # Now, mark the region of unknown with zero
# markers[unknown==255] = 0
# pplot(markers, 'markers')

# tmp = cv2.cvtColor(stripsPu[1],cv2.COLOR_GRAY2BGR)
# markers = cv2.watershed(tmp,markers)
# tmp[markers == -1] = [255,0,0]

# pplot(tmp, 'tmp')
# ============================ CRAP ============================


hUs,w = stripsPu[0].shape[:2]
blankStrip = np.zeros((hUs-dead_y,w),dtype=np.uint8)
deadzone_mask = np.concatenate((deadzoneU,blankStrip), axis=0)
# try: deadzone_mask = cv2.cvtColor(deadzone_mask, cv2.COLOR_GRAY2BGR)
# except: print("[WARNING] ------------  Unnecessary Deadzone Image Color Converting")

kernelD = cv2.getStructuringElement(cv2.MORPH_RECT,(50,5))
deadzone_mask = cv2.morphologyEx(deadzone_mask, cv2.MORPH_CLOSE, kernelD)

# print(stripsPu[0].shape, deadzone_mask.shape)
stripsPu[0] = cv2.addWeighted(stripsPu[0], 1.0, deadzone_mask, 1.0, 0)
pplot(stripsPu[0], 'stripsPu[0] After')

tmpMax = np.max(stripsPu[0])
# print(tmpMax)
dead_strip = stripsPu[0][0:14, :]
rest_strip = stripsPu[0][14:stripsPu[0].shape[0], :]

dead_strip[dead_strip < tmpMax*0.4] = 0
rest_strip[rest_strip < tmpMax*0.1] = 0
stripsPu[0] = np.concatenate((dead_strip,rest_strip), axis=0)


# pplot(stripsPu[1], 'stripsPu[1] After')



pplot(deadzone_mask, 'Deadzone Mask')
   
pUmap = np.concatenate(stripsPu, axis=0)

cnt_threshs = [55.0,50.0,80.0,40.0,40.0,40.0]
contours = []
for i, strip in enumerate(stripsPu):
    contours += vboat.find_contours(strip, cnt_threshs[i], offset=(0,hUs*i))
# contours = fCnts1 + fCnts2 + fCnts3 + fCnts4 + fCnts5 + fCnts6
print(len(contours))

disp = pUmap.copy()
for cnt in contours:
    cv2.drawContours(disp, [cnt], 0, (255,0,255), 1)
    
pplot(disp, 'Filtered Umap')

xLims, dLims, _ = vboat.extract_contour_bounds(contours)
# ==============================================================================
#                           V MAP
# ==============================================================================
threshsV = [5, 20, 40,40,40]
stripsV = strip_image(raw_vmap, nstrips=len(threshsV), horizontal_strips=False)

pplot(stripsV[0], 'stripsPv[0] Before')
# pplot(stripsV[1], 'stripsPv[1] Before')

stripsPv = []
for i, strip in enumerate(stripsV):
#     _, tmpStrip = cv2.threshold(strip, threshsV[i], 255,cv2.THRESH_BINARY)
    _, tmpStrip = cv2.threshold(strip, threshsV[i], 255,cv2.THRESH_TOZERO)
    stripsPv.append(tmpStrip)
    
tmpMax = np.max(stripsPv[0])
# print(tmpMax)
stripsPv[0][stripsPv[0] < np.max(stripsPv[0]*0.035)] = 0

top_strip = stripsPv[0][0:100, :]
bot_strip = stripsPv[0][100:stripsPv[0].shape[0], :]
    
top_strip[top_strip < tmpMax*0.1] = 0
bot_strip[bot_strip < tmpMax*0.05] = 0
stripsPv[0] = np.concatenate((top_strip,bot_strip), axis=0)

# tmpMax = np.max(stripsPv[1])
tH = stripsPv[1].shape[0]
top_strip = stripsPv[1][0:tH/2, :]
bot_strip = stripsPv[1][tH/2:tH, :]

top_strip[top_strip < tmpMax] = 0
# bot_strip[bot_strip < tmpMax*0.05] = 0
stripsPv[1] = np.concatenate((top_strip,bot_strip), axis=0)    
    
    
# pplot(stripsPv[0]**2, 'stripsPv[0] After')
# pplot(stripsPv[1]**2, 'stripsPv[1] After')
    
newV = np.concatenate(stripsPv, axis=1)
tmpV = cv2.cvtColor(newV, cv2.COLOR_GRAY2BGR)
mask, maskInv,mPxls, ground_wins,_ = vboat.get_vmap_mask(tmpV, maxStep = 15)
vmap = cv2.bitwise_and(newV,newV,mask=maskInv)
pplot(vmap*10, 'Vmap After')
# =========================================================================

obs, obsU, ybounds, dbounds, windows, nObs = vboat.find_obstacles(vmap, dLims, xLims)

dispObs = cv2.cvtColor(tImg, cv2.COLOR_GRAY2BGR)

for ob in obs:
    cv2.rectangle(dispObs,ob[0],ob[1],(150,0,0),1)
    
print(len(obs))
pplot(dispObs, 'Obstacles')
# pplot(strU1, 'stripsPu[0] Before')
# pplot(strU2, 'stripsPu[1] Before')

In [ ]:
def spread_image(image, percentile=1.1):
    copy = image.copy()
    summed = np.sum(copy, axis=1)
    total = np.sum(summed)
    percentile_limit = total / percentile
    running_total = 0
    for i in range(summed.shape[0]):
        running_total += summed[i]
        if running_total > percentile_limit:
            break

    reshape_factor = copy.shape[0] / float(i)
    print(copy.shape)
    copy = cv2.resize(copy, (copy.shape[1], int(copy.shape[0]*reshape_factor)), interpolation=cv2.INTER_NEAREST)
    print(copy.shape)
#     tmpI = int(copy.shape[0]/reshape_factor)+1
#     print(tmpI)
#     copy = copy[:tmpI,:]
#     print(copy.shape)
    return copy, reshape_factor

def despread_image(image,reshape_factor):
    copy = image.copy()
    print(copy.shape)
    sz1 = copy.shape[1]
    sz2 = int(copy.shape[0]/reshape_factor)+2
#     sz2 = int(int(copy.shape[0]/reshape_factor)*reshape_factor)
    
    copy = cv2.resize(copy, (sz1, sz2), interpolation=cv2.INTER_NEAREST)
    sz3 = int(copy.shape[0]*reshape_factor)
#     copy = copy[:sz3,:]
    print(sz1,sz2,sz3)
    return copy

In [ ]:
sprImg,fact = spread_image(strU1)
sub1 = strU1 - sprImg[:42,:]
pplot(sub1,'Spread U1')

desprdImg = despread_image(sprImg,fact)
print(desprdImg.shape)
sub2 = strU1 - desprdImg[:42,:]
pplot(sub2,'Spread U2')
pplot(desprdImg,'DeSpread U2')


In [ ]:
ellipses = []
disp = np.copy(vboat.umap_processed)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
for cnt in vboat.filtered_contours:
    ellipse = cv2.fitEllipse(cnt)
    ellipses.append(ellipse)
    cv2.ellipse(disp,ellipse,(0,255,255),1)
    
plt.figure(1)
plt.imshow(disp)
plt.show()

In [ ]:
img = np.copy(vboat.img)
print(vboat.obstacles[0])
win = vboat.obstacles[0]
print(win[0][0], win[1][0])

rectsTypeA = [] # x0, y0, x1, y1
rectsTypeB = [] # x, y, w, h
for win in vboat.obstacles:
    
    wid = win[1][0] - win[0][0]
    h = win[1][1] - win[0][1]
    cx = win[0][0] + wid/2
    cy = win[0][1] + h/2
    
    tmpA = [win[0][0], win[0][1], win[1][0], win[1][1]]
    tmpB = [cx, cy, wid, h]
    rectsTypeA.append(tmpA)
    rectsTypeB.append(tmpB)


# cv2.circle(img,(pxl[0],pxl[1]),2,(255,0,255),-1)

print(wid, h)
print(cx, cy)
print(rectsTypeB)
print(vboat.nObs)

In [ ]:
def test_intersection(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def union(a,b):
    x = min(a[0], b[0])
    y = min(a[1], b[1])
    w = max(a[0]+a[2], b[0]+b[2]) - x
    h = max(a[1]+a[3], b[1]+b[3]) - y
    return (x, y, w, h)

In [ ]:
n = vboat.nObs
idcs = []
pick = []
boxes = np.array(rectsTypeA)
x1 = boxes[:,0]
y1 = boxes[:,1]
x2 = boxes[:,2]
y2 = boxes[:,3]
area = (x2 - x1 + 1) * (y2 - y1 + 1)
idxs = np.argsort(y2)
overlapThresh = 0.05
supressed = []
top = []
non = []
print(idxs)
while len(idxs) > 0:
    # grab the last index in the indexes list, add the index
    # value to the list of picked indexes, then initialize
    # the suppression list (i.e. indexes that will be deleted)
    # using the last index
    last = len(idxs) - 1
    i = idxs[last]
    pick.append(i)
    suppress = [last]
    # loop over all indexes in the indexes list
    print(i)
    overlaps = False
    for pos in xrange(0, last):
        # grab the current index
        j = idxs[pos]

        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = max(x1[i], x1[j])
        yy1 = max(y1[i], y1[j])
        xx2 = min(x2[i], x2[j])
        yy2 = min(y2[i], y2[j])

        # compute the width and height of the bounding box
        w = max(0, xx2 - xx1 + 1)
        h = max(0, yy2 - yy1 + 1)

        # compute the ratio of overlap between the computed
        # bounding box and the bounding box in the area list
        
        overlap = float(w * h) / area[j]

        print("Window %d Overlaps %.4f with Rectangle %d" % (j,overlap, i) )
        # if there is sufficient overlap, suppress the
        # current bounding box
        if overlap > overlapThresh:
#             suppress.append(pos)
            supressed.append(pos)
            top.append([pos,i])
            overlaps = True

    # delete all indexes from the index list that are in the
    # suppression list
    print("suppressing idxs", suppress)
#     supressed.append(suppress)
    idxs = np.delete(idxs, suppress)
    if(not overlaps):
        non.append(last)

rectangles = [] 
for a,b in top:
    rect = union(rectsTypeA[a],rectsTypeA[b])
    rectangles.append(rect)
    

print(top)
print(non)
print(supressed)
mask = np.isin(non, supressed, invert=True)
remain = np.where(mask)
print(remain)
rectangles.append(rectsTypeA[remain[0][0]])

print(rectangles)
for i in range(n):
    has_overlap = False
    for j in range(n):
        if(i is not j):
            iou = test_intersection(rectsTypeA[i],rectsTypeA[j])
            if(iou >= 0.05):
                has_overlap = True
                idcs.append([i, j])
                print("IoU for Rect %d and Rect %d = %.3f" % (i, j, iou))
    if(not has_overlap):
        print("Rect %d has no overlapping segments" % i)

In [ ]:
disp = np.copy(vboat.img)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
for x1,y1,x2,y2 in boxes[pick]:
    cv2.rectangle(disp,(x1,y1),(x2,y2),(0,255,255), 1)
    
plt.figure(10)
plt.imshow(disp)
plt.show()

In [ ]:
disp = np.zeros_like(vboat.img)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
masks = []
for win in vboat.obstacles:
    mask = np.copy(disp)
    cv2.rectangle(mask,win[0],win[1],(255,255,255), cv2.FILLED)
    mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
    masks.append(mask)
    

tmp = cv2.bitwise_and(vboat.img,vboat.img, mask=masks[0])

tmpMean = np.mean(tmp)
print(tmpMean/0.001)
plt.figure(2)
plt.imshow(tmp)
plt.show()